In [ ]:
import azureml.dataprep as dprep
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
clicks_df = pd.read_csv("../clicks_latest_raw.csv")

In [ ]:
clicks_df.head()

In [ ]:
len(clicks_df) #Total Clicks

In [ ]:
clicks_df.isna().sum() #All Null Values

In [ ]:
clicks_df = clicks_df.dropna()

In [ ]:
clicks_df.isna().sum() #All Null Values

In [ ]:
print(len(clicks_df)) #After removing null values

In [ ]:
max_time = clicks_df['timeframe'].max()

In [ ]:
split_day = max_time - 86400 * 1

In [ ]:
# Make train and test data have the same vehicles

test_df = clicks_df.loc[clicks_df['timeframe'] >= split_day]

test_df['reg_no'].nunique() #No. of unique vehicles in test data

In [ ]:
train_df = clicks_df.loc[clicks_df['timeframe'] < split_day]
train_df['reg_no'].nunique() #No. of unique vehicles in train data

In [ ]:
#Filter both train and test data - remove vehicles in train data that are not in test data

train_df = train_df.loc[train_df['reg_no'].isin(test_df['reg_no'])]
train_df['reg_no'].nunique()

In [ ]:
clicks_df = pd.concat([train_df, test_df]) #Use new clicks df
print(len(clicks_df))

In [ ]:
cols = ["session_id","reg_no"]
clicks_df = clicks_df.loc[(clicks_df[cols].shift() != clicks_df[cols]).any(axis=1)] # Remove Consecutive same clicks in session
print(len(clicks_df)) #After removing consecutive duplicates
clicks_df.head()

In [ ]:
clicks_df["reg_no"].nunique() #Unique Cars Seen

In [ ]:
clicks_df["session_id"].nunique() #Unique Sessions

In [ ]:
clicks_df["client_id"].nunique() #Unique Users

In [ ]:
grouped = clicks_df.groupby('session_id')['page'].agg(["count"])
print(grouped.head())

In [ ]:
filtered = grouped.query('count>1 and count <40')

x = len(filtered)

print(f" Sessions with greater than 1 click and less than 40 clicks = {x}")
print(f" Sessions with only 1 click or more than 40 = {len(grouped)-x}")

In [ ]:
total_clicks = filtered['count'].sum()
print(f" Total clicks after filtering = {total_clicks}")

In [ ]:
average_clicks = filtered['count'].mean()
print(f" Average clicks per session = {average_clicks}")

In [ ]:
max_clicks = filtered['count'].max()
min_clicks = filtered['count'].min()
print(f"Max clicks per session = {max_clicks}")
print(f"Min clicks per session = {min_clicks}")

In [ ]:
filtered.sort_values(by='count', ascending=False).head()

In [ ]:
filtered.plot.hist(bins=15, alpha=0.5) #Clicks per session frequency

In [ ]:
clean_df = pd.merge(clicks_df, filtered, on='session_id')
clean_df.head()

In [ ]:
clean_df.drop('count', axis=1, inplace=True)
print(len(clean_df))

In [ ]:
clean_df.to_csv("../clicks_clean_latest.csv", index=False) #Save the cleaned data